In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # Demo 3: SQL Server コンテナ内のUpgrade（2017->2019）
 ## 3-1.SQL Server 2019イメージをPull（事前にPull済み）

In [15]:
%%bash
sudo docker pull mcr.microsoft.com/mssql/server:2019-RC1
sudo docker image list | grep 2019-RC1


2019-RC1: Pulling from mssql/server
Digest: sha256:a11facbda1b1cc299d4a37499ef79cd18e38bfb8e5dd7e45cc73670cc07772e5
Status: Image is up to date for mcr.microsoft.com/mssql/server:2019-RC1
mcr.microsoft.com/mssql/server:2019-RC1
mcr.microsoft.com/mssql/server   2019-RC1            c5a295efea97        2 months ago        1.5GB


 ## 3-2.SQL Server2019コンテナ起動（SQL Server 2017で作成したデータファイルをマウント）

In [16]:
%%bash
sudo docker run -e ACCEPT_EULA=Y -e MSSQL_SA_PASSWORD=$PASSWORD -p 11433:1433 -v ~/SQL_Server_Container/DataVolume:/var/opt/mssql --name sql19rc1 -d mcr.microsoft.com/mssql/server:2019-RC1
sudo docker ps -a --filter name=sql19rc1
sleep 30


da9292072c1af86639de6c9d527b2aca475cf8afaa33062aa7798f1b5524b29c
CONTAINER ID        IMAGE                                     COMMAND                  CREATED             STATUS                  PORTS                     NAMES
da9292072c1a        mcr.microsoft.com/mssql/server:2019-RC1   "/opt/mssql/bin/perm…"   2 seconds ago       Up Less than a second   0.0.0.0:11433->1433/tcp   sql19rc1


 ## 3-3.SQL Server に接続すると”Upgrade中だから待て”のエラー
 ### Sqlcmd: Error: Microsoft ODBC Driver 17 for SQL Server : Login failed for user 'sa'. Reason: Server is in script upgrade mode. Only administrator can connect at this time..

In [17]:
%%bash
sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P $PASSWORD -Q 'select @@version'


Sqlcmd: Error: Microsoft ODBC Driver 17 for SQL Server : Login failed for user 'sa'. Reason: Server is in script upgrade mode. Only administrator can connect at this time..


CalledProcessError: Command 'b"sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P $PASSWORD -Q 'select @@version'\n"' returned non-zero exit status 1.

 ## 3-4.アップグレードが終われば接続できるようになる(数分かかる)
 ### Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64)

In [18]:
%%bash
sudo docker exec sql19rc1 /opt/mssql-tools/bin/sqlcmd -S localhost -U sa -P $PASSWORD -Q 'select @@version'



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


 ## 3-5.SQL Server 2017から2019にアップグレードしたDBをSELECT

In [19]:
%%bash
sqlcmd -S localhost,11433 -U sa -P $PASSWORD -Q 'use DevDB; select * from tab01'


Changed database context to 'DevDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on Docker!                                                                                                                                                                                                                                      

(1 rows affected)
